# Quora Insincere Questions Classification

## Detect toxic content to improve online conversations

## Problem

* **Handle toxic and disivie content / miseleading content**

* **To do that we have to develop a model that identify and flag insincere questions**

## Evaluation

* **For each qid in the testset, predict the corresponding questions_text:**

 * **Is Insincere => 1**

 * **Is Sincere => 0**

* **Submissions are evaluated on F1 Score between the predicted and the observed targets**

## Data Fields

* **qid : unique question identifer**

* **question_text : Quora question text**

* **target : a question labeled << insincere >> has a value of 1, otherwise 0**

In [ ]:
import os
import numpy as np 
import pandas as pd 

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import nltk
from sklearn.pipeline import Pipeline
from nltk.corpus import stopwords
from string import punctuation 
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn import model_selection
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

In [ ]:
import warnings
warnings.filterwarnings("ignore")

## Load the dataset

In [ ]:
path = "../input"
df_train = pd.read_csv(os.path.join(path, "train.csv"))
df_test = pd.read_csv(os.path.join(path, "test.csv"))

In [ ]:
df_train.shape

In [ ]:
df_train.head()

In [ ]:
df_test.shape

In [ ]:
df_test.head()

In [ ]:
df_train["target"].value_counts()

## Exploratory Data Analysis

In [ ]:
insincere = df_train[df_train["target"] == 1]
sincere = df_train[df_train["target"] == 0]

In [ ]:
sincere.head()

In [ ]:
insincere.head()

### Distribution of Sincere / Insincere questions

In [ ]:
question_class = df_train["target"].value_counts()
question_class.plot(kind= "bar", color= ["blue", "orange"])
plt.title("Bar chart")
plt.show()

In [ ]:
print(df_train["target"].value_counts())
print(sum(df_train["target"] == 1) / sum(df_train["target"] == 0) * 100, "percent of questions are insincere.")
print(100 - sum(df_train["target"] == 1) / sum(df_train["target"] == 0) * 100, "percent of questions are sincere")

**We have a Unbalenced Data**

### Insincere Word cloud

In [ ]:
from wordcloud import WordCloud, STOPWORDS
stopwords = set(STOPWORDS)

# Generate a word cloud image
sincere_wordcloud = WordCloud(width=600, height=400).generate(str(sincere["question_text"]))
#Positive Word cloud
plt.figure(figsize=(10,8), facecolor="black")
plt.imshow(sincere_wordcloud)
plt.axis("off")
plt.tight_layout(pad=0)
plt.show();

### Insincere Word cloud

In [ ]:
from wordcloud import WordCloud, STOPWORDS
stopwords = set(STOPWORDS)

# Generate a word cloud image
insincere_wordcloud = WordCloud(width=600, height=400).generate(str(insincere["question_text"]))
#Positive Word cloud
plt.figure(figsize=(10,8), facecolor="black")
plt.imshow(insincere_wordcloud)
plt.axis("off")
plt.tight_layout(pad=0)
plt.show();

## Feature Engineering

### Number of words in the text

In [ ]:
df_train["number_words"] = df_train["question_text"].apply(lambda x: len(x.split()))
df_test["number_words"] = df_test["question_text"].apply(lambda x: len(x.split()))

### Number of unique words in the text

In [ ]:
df_train["num_unique_words"] = df_train["question_text"].apply(lambda x: len(set(str(x).split())))
df_test["num_unique_words"] = df_test["question_text"].apply(lambda x: len(set(str(x).split())))

### Number of characters in the text

In [ ]:
df_train["num_chars"] = df_train["question_text"].apply(lambda x: len(str(x)))
df_test["num_chars"] = df_test["question_text"].apply(lambda x: len(str(x)))

### Number of Stopwords in text

In [ ]:
from nltk.corpus import stopwords 
stop_words = set(stopwords.words("english"))

In [ ]:
df_train["num_stopwords"] = df_train["question_text"].apply(lambda x : len([nw for nw in str(x).split() if nw.lower() in stop_words]))
df_test["num_stopwords"] = df_test["question_text"].apply(lambda x : len([nw for nw in str(x).split() if nw.lower() in stop_words]))

### Number of punctuations in text

In [ ]:
df_train["num_punctuation"] = df_train["question_text"].apply(lambda x : len([np for np in str(x) if np in punctuation]))
df_test["num_punctuation"] = df_test["question_text"].apply(lambda x : len([np for np in str(x) if np in punctuation]))

### Number of Upper case and Lower case in text

In [ ]:
df_train["num_uppercase"] = df_train["question_text"].apply(lambda x : len([nu for nu in str(x).split() if nu.isupper()]))
df_test["num_uppercase"] = df_test["question_text"].apply(lambda x : len([nu for nu in str(x).split() if nu.isupper()]))

In [ ]:
df_train["num_lowercase"] = df_train["question_text"].apply(lambda x : len([nl for nl in str(x).split() if nl.islower()]))
df_test["num_lowercase"] = df_test["question_text"].apply(lambda x : len([nl for nl in str(x).split() if nl.islower()]))

### Number of title in text

In [ ]:
df_train["num_title"] = df_train["question_text"].apply(lambda x : len([nl for nl in str(x).split() if nl.istitle()]))
df_test["num_title"] = df_test["question_text"].apply(lambda x : len([nl for nl in str(x).split() if nl.istitle()]))

In [ ]:
df_train[df_train["target"] == 1].describe()

In [ ]:
df_train[df_train["target"] == 0].describe()

### Box plot of number of words according to the target

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(12, 10)
sns.boxplot(data=df_train, y="number_words", x="target",orient="v")
ax.set(xlabel="target", ylabel="number of words", title="Box plot of number of words according to the target");

### Box plot of number of unique words according to the target

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(12, 10)
sns.boxplot(data=df_train, y="num_unique_words", x="target",orient="v")
ax.set(xlabel="target", ylabel="number of unique words", title="Box plot of number of unique words according to the target");

### Box plot of number of characters according to the target

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(12, 10)
sns.boxplot(data=df_train, y="num_chars", x="target",orient="v")
ax.set(xlabel="target", ylabel="number of characters", title="Box plot of number of characters according to the target");

### Box plot of number of stopwords according to the target

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(12, 10)
sns.boxplot(data=df_train, y="num_stopwords", x="target",orient="v")
ax.set(xlabel="target", ylabel="number of stopwords", title="Box plot of number of stopwords according to the target");

### Box plot of number of lowercases according to the target

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(12, 10)
sns.boxplot(data=df_train, y="num_lowercase", x="target",orient="v")
ax.set(xlabel="target", ylabel="number of lowercases", title="Box plot of number of lowercases according to the target");

### Box plot of number of titles according to the target

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(12, 10)
sns.boxplot(data=df_train, y="num_title", x="target",orient="v")
ax.set(xlabel="target", ylabel="number of titles", title="Box plot of number of titles according to the target");

## Remove Stopwords and Punctuation

In [ ]:
def text_process(question):
    nopunc = [char for char in question if char not in punctuation]
    nopunc = "".join(nopunc)
    meaning = [word for word in nopunc.split() if word.lower() not in stopwords.words("english")]
    return( " ".join( meaning ))

In [ ]:
clean_train = df_train["question_text"].apply(text_process)

In [ ]:
clean_train.head()

In [ ]:
clean_test = df_test["question_text"].apply(text_process)

In [ ]:
clean_test.head()

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(clean_train, df_train.target.values, test_size=0.2, stratify = df_train.target.values)

In [ ]:
X_train.shape, X_val.shape

In [ ]:
y_train.shape, y_val.shape

## Model Building

In [ ]:
pipeline = Pipeline([("cv",CountVectorizer(analyzer="word",ngram_range=(1,4),max_df=0.9)),
                     ("clf",LogisticRegression(solver="saga", class_weight="balanced", C=0.45, max_iter=250, verbose=1))])

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(clean_train, df_train.target.values, test_size=0.1, stratify = df_train.target.values)

In [ ]:
lr_model = pipeline.fit(X_train, y_train)

In [ ]:
lr_model

In [ ]:
y_pred = lr_model.predict(X_val)

## Evaluating Results

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
cm = confusion_matrix(y_val, y_pred)
sns.heatmap(cm, cmap="Blues", annot=True, square=True, fmt=".0f");

In [ ]:
print(classification_report(y_val, y_pred))

In [ ]:
y_pred_final = pipeline.predict(clean_test)
y_pred_final

## Formatting Final Results

In [ ]:
df_sub = pd.DataFrame({"qid":df_test["qid"], "prediction":y_pred_final})
df_sub.head()

In [ ]:
df_sub.to_csv('submission.csv', index=False)